In [3]:
from EasyOCR.easyocr import easyocr
from glob import glob
import cv2
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import json

/root/anaconda3/envs/awsec2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
r = easyocr.Reader(lang_list=['ko', 'en'], gpu=True)#, recog_network='trocr')

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [19]:
!pip install gdown

In [20]:
!gdown https://fontmeme.com/fonts/download/243163/nanum-gothic.zip

Downloading...
From: https://fontmeme.com/fonts/download/243163/nanum-gothic.zip
To: /data/SKT_FLY_AI_3/ec2_server_test/nanum-gothic.zip
100%|██████████████████████████████████████| 6.74M/6.74M [00:01<00:00, 5.71MB/s]


In [21]:
!unzip -o nanum-gothic.zip

Archive:  nanum-gothic.zip
  inflating: OFL.txt                 
  inflating: NanumGothic-Regular.ttf  
  inflating: NanumGothic-Bold.ttf    
  inflating: NanumGothic-ExtraBold.ttf  


In [65]:
def draw_boxes_with_pillow_text(image, detections):
    img_y_max, img_x_max = image.shape[0], image.shape[1]
    def normal(i):
        return_value = 0
        if normal.counter:
            return_value = int(i*img_x_max)
        else:
            return_value = int (i*img_y_max)
        normal.counter = not normal.counter
        return return_value
    normal.counter = True
    image_clone = image.copy()
    for detection in detections:
        if type(detection) == str:
            data = json.loads(detection)
            points = [list(map(normal, i)) for i in data['boxes']]
            back_color = data['background_color']
            text_color = tuple(data['text_color'])
            text = data['text']
        else:
            points = detection[0]
            back_color = 0xFFFFFF
            text_color = 0x000000
            text = detection[1]
        font_size = 20    
        print (points, text)
        # 상자를 위한 좌표 추출
        rect = cv2.boundingRect(np.array(points, dtype=np.int32))
        (x, y, w, h) = rect
        #color = tuple(map(int, image_clone[y, x, :3]))
        image_clone = cv2.rectangle(img = image_clone, pt1 = (x, y), pt2 = (x+w, y+h), color = back_color, thickness = -1)
        # # 텍스트 그리기 (Pillow)
        # pil_img = Image.fromarray(cv2.cvtColor(image_clone, cv2.COLOR_BGR2RGB))
        # draw = ImageDraw.Draw(pil_img)
        # font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"  # Linux 예시 경로
        # font = ImageFont.truetype(font_path, font_size)
        # print (text_color)
        # 텍스트 그리기 (Pillow)
        pil_img = Image.fromarray(cv2.cvtColor(image_clone, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(pil_img)
        font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"  # Linux 예시 경로
        
        # 초기 폰트 크기
        font_size = 20  
        font = ImageFont.truetype(font_path, font_size)
        text_width, text_height = draw.textsize(text, font=font)

        # 폰트 크기 조절: bounding box의 크기에 맞게 텍스트 크기를 조절
        while text_width > w or text_height > h:
            font_size -= 1  # 폰트 크기 감소
            font = ImageFont.truetype(font_path, font_size)
            text_width, text_height = draw.textsize(text, font=font)
            if font_size <= 10:  # 최소 폰트 크기 설정
                break

        # 텍스트의 시작 위치 계산
        text_start_x = x + (w - text_width) / 2  
        text_start_y = y + (h - text_height) / 2  

        draw.text((text_start_x, text_start_y), text, font=font, fill=text_color[:3])

        text_width, text_height = draw.textsize(text, font=font)
        #draw.text((x + w/2., y + h/2.), text, font=font, fill=text_color[:3], align='center')

        # 이미지를 다시 OpenCV 형태로 변환
        image_clone[:,:,:3] = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
            #image_clone[:, :,3] = 1
    # 결과 이미지 출력
    return image_clone

In [67]:
src = cv2.imread('test_kakao.jpg')
result = r.readtext(image = src,batch_size=10, min_size=5, output_format='json_specific_and_relative_pos')
print (result)
img = draw_boxes_with_pillow_text(src, result)
cv2.imwrite('img.jpg', img)

Total Detection 걸린 시간 0.4665820598602295


/root/anaconda3/envs/awsec2/lib/python3.9/site-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Total Recognize 걸린 시간 1.0469329357147217
['{"boxes": [[0.11475409836065574, 0.07890070921985816], [0.2693208430913349, 0.07890070921985816], [0.2693208430913349, 0.11790780141843972], [0.11475409836065574, 0.11790780141843972]], "text": "MeetUS 번역", "confident": 0.9, "background_color": [217, 202, 183, 127], "text_color": [61, 53, 52, 255]}', '{"boxes": [[0.117096018735363, 0.12411347517730496], [0.15768930523028885, 0.12411347517730496], [0.15768930523028885, 0.1524822695035461], [0.117096018735363, 0.1524822695035461]], "text": "요 5", "confident": 0.9, "background_color": [220, 203, 184, 127], "text_color": [159, 144, 130, 255]}', '{"boxes": [[0.20921155347384857, 0.20921985815602837], [0.27946916471506633, 0.20921985815602837], [0.27946916471506633, 0.2375886524822695], [0.20921155347384857, 0.2375886524822695]], "text": "ERROR", "confident": 0.9, "background_color": [59, 226, 243, 127], "text_color": [24, 96, 104, 255]}', '{"boxes": [[0.2068696330991413, 0.26861702127659576], [0.68

/tmp/ipykernel_856741/320622864.py:46: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
/tmp/ipykernel_856741/320622864.py:62: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
/tmp/ipykernel_856741/320622864.py:46: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
/tmp/ipykernel_856741/320622864.py:62: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
/tmp/ipykernel_856741/320622864.py:46: DeprecationWarning: textsize is deprecated and will be removed in

True